**Cecilia** **Conde**


```
Scrip clase ternaria
```



# Librerías


In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Couldn't find program: 'bash'


Configuracion la extensión de %sql para el notebook, lo que nos va a permitir usar lenguaje SQL directamente sobre una celda

In [2]:
import duckdb as duckdb
import pandas as pd

# %load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [3]:
con = duckdb.connect()

 # Datos

In [4]:
#seteo carpeta de trabajo y archivo
dataset_path = 'C:/Users/mconde/dmeyf2024/Competencia_03/'
# dataset_path = '~/buckets/b1/datasets' ojo esto cuando tenga colab en la nube
dataset_file = 'competencia_03_crudo.csv.gz'

In [6]:
%%sql
create or replace table competencia_03_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Visualización de tabla de sql

In [7]:
%%sql
select *
from competencia_03_crudo limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1,0.0,-44919.57,3.23,7136,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1,0.0,-30184.75,15.24,393,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1,0.0,-3730.14,0.00,2755,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1,0.0,-13196.25,0.00,8194,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1,0.0,-22073.65,0.00,7876,21902.85,6.0,0.0,891.48
5,249236712,201901,1,0,1,42,1,1746.17,1746.17,0.00,...,NaN,1,0.0,NaN,NaN,10,NaN,NaN,NaN,0.00
6,249237079,201901,1,0,1,62,117,19193.25,110961.55,3147.21,...,0.0,8,0.0,-3688.58,0.00,3531,257.94,1.0,0.0,586.50
7,249237446,201901,1,0,1,52,253,3268.57,20250.17,220.94,...,0.0,8,0.0,-5574.87,0.00,3222,-1.34,4.0,0.0,680.34
8,249239632,201901,1,0,1,50,239,990.51,8868.52,343.85,...,0.0,1,0.0,-18110.05,0.00,5054,1596.32,2.0,0.0,1114.35
9,249242331,201901,1,0,1,53,256,3429.46,27997.33,1153.68,...,0.0,1,0.0,-49544.20,116.27,5708,35766.21,18.0,0.0,2498.49


 # Función de Clase ternaria


In [ ]:
#este es el que hay que usar con los parametros del profe!!

In [8]:


%%sql
create or replace table competencia_03 as
with periodos as (
    select distinct foto_mes from competencia_03_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_03_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if (mes_2 = 1, 'CONTINUA',
          if(mes_1 = 0 , 'BAJA+1',
          if(mes_2= 0,'BAJA+2', null))) as clase_ternaria
    from todo t
    left join competencia_03_crudo c using (numero_de_cliente, foto_mes)
)
select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1;

-- Agregar el conteo de clase ternaria
select clase_ternaria, count(*) as cantidad_clientes
from competencia_03
group by clase_ternaria;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,clase_ternaria,cantidad_clientes
0,CONTINUA,4528247
1,BAJA+1,21965
2,None,329790
3,BAJA+2,21235


Revisamos que todo salga como esperamos

In [9]:
%sql select count(*) from competencia_03

,count_star()
0,4901237


Y vemos la cardinalidad de las clases por periodo

In [10]:
%%sql
PIVOT competencia_03
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [11]:
%%sql
select * from competencia_03 limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249619949,201901,1,0,1,43,44,2244.24,21440.00,673.26,...,1,0.0,-12305.89,0.00,903,8423.40,16,0,1067.43,CONTINUA
1,249619949,201902,1,0,1,43,45,1492.09,21807.50,751.06,...,1,0.0,-16474.52,0.00,931,78633.23,31,0,4914.87,CONTINUA
2,249619949,201903,1,0,1,43,46,3920.09,24215.59,711.90,...,4,0.0,-104568.84,1583.99,962,22431.65,22,0,1818.15,CONTINUA
3,249619949,201904,1,0,1,43,47,2926.99,24922.94,1697.33,...,-1,0.0,-30941.08,0.00,992,15812.69,16,0,2322.54,CONTINUA
4,249619949,201905,1,0,1,43,48,0.00,0.00,0.00,...,2,0.0,-31195.45,11.71,1023,8151.16,10,0,2181.78,CONTINUA
5,249619949,201906,1,0,1,43,49,2277.87,25221.99,-14.60,...,4,0.0,-26135.53,0.00,1053,5030.22,7,0,2029.29,CONTINUA
6,249619949,201907,1,0,1,43,50,1031.13,25030.36,128.44,...,0,0.0,-22884.93,4.70,1084,9911.74,14,2,15342.84,CONTINUA
7,249619949,201908,1,0,1,43,51,2098.75,26441.30,2640.71,...,3,0.0,-48975.94,0.00,1115,28831.42,17,0,4011.66,CONTINUA
8,249619949,201909,1,0,1,44,52,3426.62,26990.67,3575.63,...,5,0.0,-56974.00,0.00,1145,102721.78,18,1,20034.84,CONTINUA
9,249619949,201910,1,0,1,44,53,0.00,0.00,0.00,...,1,0.0,-140526.40,1407.94,1176,32018.13,19,0,3870.90,CONTINUA


Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [12]:
print(dataset_path)

C:/Users/mconde/dmeyf2024/Competencia_03/


In [14]:
%%sql
COPY competencia_03 TO 'C:/Users/mconde/dmeyf2024/Competencia_03/competencia_3.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
